<a href="https://colab.research.google.com/github/barudenko/projects/blob/main/dota_2_pro_matches_research/dota_2_data_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Анализ профессиональных матчей Dota 2

**Данные**

[Источник данных](https://www.kaggle.com/datasets/bwandowando/dota-2-pro-league-matches-2023/data)

Данные актуальные, выкачиваются с kaggle, разархивируются и объединяются в целые таблицы. Далее оставляются только потенциально интересные для исследования признаки, из них формируется локальная база данных SQLite. Дальнейшая работа производится с ней.

Итоговые данные состоят из 9 таблиц  

Основные:
- `matches` - информация о матчах
- `players` - информация об игроках и их активностях
- `teams` - информация о командах
- `picks_bans` - информация о стадии пиков/банов героев в каждом матче

Вспомогательные:
- `heroes` - инфо о героях
- `items` - инфо о предметах
- `leagues` - инфо о профессиональных лигах
- `patch` - инфо о патчах (версиях) игры
- `regions` - инфо о регионе сервера матча

**Цель исследования**

Посмотреть, что происходит на профессиональной сцене Dota 2 в последнем патче: посмотреть на героев, предметы, их винрейт, на длительность матча и время первого фб. И посмотреть на некоторые командные показатели, и определить, влияют ли они на победу в матче, или нет.

In [1]:
%%capture
!pip install kaggle

In [2]:
import os
import shutil
import warnings
import pandas as pd
import numpy as np

import sqlite3

from scipy.stats import chi2_contingency
from scipy.stats import ttest_ind

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

from google.colab import userdata
import gdown

In [3]:
#отображение больших чисел полностью
pd.set_option('display.float_format', '{:.2f}'.format)

#отключение текста предупреждений pandas
pd.options.mode.chained_assignment = None

#отключение текста предупреждений DtypeWarning
warnings.filterwarnings("ignore", category=pd.errors.DtypeWarning)

#параметры отображения графиков
# pio.renderers.default='notebook'
# pio.renderers.default='colab'

# Подготовка данных и создание базы

## Скачивание и объединение

In [5]:
kaggle_gdrive_id = userdata.get('kaggle_gdrive_id')

In [6]:
%%capture
gdown.download(id=kaggle_gdrive_id) # kaggle.json

In [7]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [8]:
!kaggle datasets download -d bwandowando/dota-2-pro-league-matches-2023

Dataset URL: https://www.kaggle.com/datasets/bwandowando/dota-2-pro-league-matches-2023
License(s): CC0-1.0
100% 6.17G/6.18G [00:54<00:00, 139MB/s]
100% 6.18G/6.18G [00:54<00:00, 122MB/s]


In [9]:
%%capture
!unzip dota-2-pro-league-matches-2023

In [10]:
!rm dota-2-pro-league-matches-2023.zip

In [11]:
os.getcwd()

'/content'

In [12]:
os.listdir()

['.config',
 '2023',
 '2019',
 '202403',
 '202407',
 '202406',
 '2020',
 '202409',
 '202402',
 '202405',
 '202408',
 '202401',
 '202404',
 '2016',
 'Images',
 '2022',
 '2018',
 'Constants',
 '2017',
 '2021',
 'sample_data']

In [13]:
# Указываем путь к рабочей директории
base_dir = '/content'

# Список папок, которые не нужно обрабатывать
ignore_folders = ['Images', 'Constants', 'sample_data', '.config']

# Получаем список папок годов, отсортированных в порядке возрастания
year_folders = sorted([folder for folder in os.listdir(base_dir) if folder not in ignore_folders])

# Директория для сохранения объединенных файлов
output_dir = os.path.join(base_dir, 'merged_data')
os.makedirs(output_dir, exist_ok=True)

# Словарь для хранения эталонных заголовков для каждого типа файла
reference_columns = {}

# Функция для поэтапного сохранения данных
def append_to_csv(dataframe, file_path):
    """Добавляет DataFrame в CSV файл. Если файл не существует, создает новый."""
    if not os.path.exists(file_path):
        dataframe.to_csv(file_path, index=False)
    else:
        dataframe.to_csv(file_path, mode='a', header=False, index=False)

# Функция для приведения колонок в чанке к эталонным
def align_columns(base_columns, chunk):
    """
    Приводим колонки в чанке к базовым колонкам: добавляем отсутствующие и удаляем лишние.
    """
    # Добавляем отсутствующие столбцы и заполняем их значением NaN
    for col in base_columns:
        if col not in chunk.columns:
            chunk[col] = pd.NA
    # Удаляем лишние столбцы
    chunk = chunk[base_columns]
    return chunk

# 1. Определяем эталонные заголовки и сортируем файлы по размеру на основе файлов из 2016 года
reference_folder = os.path.join(base_dir, '2016')
sorted_files = []

if os.path.exists(reference_folder):
    # Проверяем и сортируем файлы по размеру
    for csv_file in os.listdir(reference_folder):
        file_path = os.path.join(reference_folder, csv_file)
        if os.path.isfile(file_path) and csv_file.endswith('.csv'):
            # Добавляем файл и его размер в список
            sorted_files.append((csv_file, os.path.getsize(file_path)))

    # Сортируем список файлов по размеру
    sorted_files = sorted(sorted_files, key=lambda x: x[1])

    # Считываем эталонные заголовки для каждого CSV файла
    for csv_file, _ in sorted_files:
        reference_file_path = os.path.join(reference_folder, csv_file)

        # Если файл существует, читаем только заголовки
        if os.path.exists(reference_file_path):
            reference_df = pd.read_csv(reference_file_path, nrows=1)
            reference_columns[csv_file] = reference_df.columns  # Сохраняем заголовки

# 2. Этап объединения данных
# Проходим по каждому отсортированному файлу (начиная с самого легкого)
for csv_file, _ in sorted_files:
    print(f"Объединение файла: {csv_file}")
    output_path = os.path.join(output_dir, f'merged_{csv_file}')

    # Проходим по каждой папке года
    for year_folder in year_folders:
        year_folder_path = os.path.join(base_dir, year_folder)
        csv_file_path = os.path.join(year_folder_path, csv_file)

        # Если файл существует в данной папке года
        if os.path.exists(csv_file_path):
            chunk_size = 100000  # Размер чанка для чтения
            for chunk in pd.read_csv(csv_file_path, chunksize=chunk_size):
                # Получаем эталонные колонки для текущего типа файла
                base_columns = reference_columns[csv_file]

                # Приводим колонки в текущем чанке к базовым колонкам
                chunk = align_columns(base_columns, chunk)

                # Добавляем чанк в итоговый CSV файл
                append_to_csv(chunk, output_path)

            # Удаляем исходный CSV файл после обработки
            os.remove(csv_file_path)
            print(f"Удален файл: {csv_file_path}")

# 3. Этап удаления пустых папок после обработки всех файлов
for year_folder in year_folders:
    year_folder_path = os.path.join(base_dir, year_folder)

    # Проверяем, пуста ли папка, и удаляем её, если пуста
    if os.path.isdir(year_folder_path) and not os.listdir(year_folder_path):
        os.rmdir(year_folder_path)
        print(f"Удалена пустая папка: {year_folder_path}")

print("----------\nОбъединение завершено.\nИсходные файлы и пустые папки удалены.\nИтоговые датасеты сохранены в директорию 'merged_data'.")

Объединение файла: teams.csv
Удален файл: /content/2016/teams.csv
Удален файл: /content/2017/teams.csv
Удален файл: /content/2018/teams.csv
Удален файл: /content/2019/teams.csv
Удален файл: /content/2020/teams.csv
Удален файл: /content/2021/teams.csv
Удален файл: /content/2022/teams.csv
Удален файл: /content/2023/teams.csv
Удален файл: /content/202401/teams.csv
Удален файл: /content/202402/teams.csv
Удален файл: /content/202403/teams.csv
Удален файл: /content/202404/teams.csv
Удален файл: /content/202405/teams.csv
Удален файл: /content/202406/teams.csv
Удален файл: /content/202407/teams.csv
Удален файл: /content/202408/teams.csv
Удален файл: /content/202409/teams.csv
Объединение файла: main_metadata.csv
Удален файл: /content/2016/main_metadata.csv
Удален файл: /content/2017/main_metadata.csv
Удален файл: /content/2018/main_metadata.csv
Удален файл: /content/2019/main_metadata.csv
Удален файл: /content/2020/main_metadata.csv
Удален файл: /content/2021/main_metadata.csv
Удален файл: /con

In [14]:
os.listdir()

['.config', 'Images', 'merged_data', 'Constants', 'sample_data']

In [15]:
os.chdir('/content/merged_data/')

In [16]:
csv_names = [name.replace('merged_', '') for name in os.listdir()]
for merged_name, new_name in zip(os.listdir(), csv_names):
  os.rename(merged_name, new_name)

In [17]:
os.listdir()

['teamfights.csv',
 'objectives.csv',
 'cosmetics.csv',
 'main_metadata.csv',
 'chat.csv',
 'all_word_counts.csv',
 'radiant_exp_adv.csv',
 'picks_bans.csv',
 'draft_timings.csv',
 'players.csv',
 'teams.csv',
 'radiant_gold_adv.csv']

In [18]:
os.path.getsize(f"/content/merged_data/{sorted_files[-1][0]}")

29232920866

Самая тяжелая таблица весит ~27 гб

## Подготовка таблиц и создание БД

In [19]:
os.chdir('/content/')

In [20]:
# Создаем подключение к базе данных (или создаем файл базы данных, если его нет)
conn = sqlite3.connect('dota_pro_matches.db')
c = conn.cursor()

### Основные таблицы

In [21]:
t = pd.read_csv(
    '/content/merged_data/players.csv',
    usecols=[
        'match_id',
        'account_id',
        'personaname',
        'hero_id',
        'isRadiant',
        'win',
        'radiant_win',
        'kills',
        'deaths',
        'assists',
        'last_hits',
        'denies',
        'neutral_kills',
        'lane_kills',
        'hero_damage',
        'hero_healing',
        'tower_damage',
        'level',
        'gold_per_min',
        'xp_per_min',
        'actions_per_min',
        'item_0',
        'item_1',
        'item_2',
        'item_3',
        'item_4',
        'item_5',
        'item_neutral',
        'backpack_0',
        'backpack_1',
        'backpack_2',
        'observer_uses',
        'sentry_uses',
        'purchase_gem',
        'purchase_rapier',
        'firstblood_claimed',
        'gold_spent',
        'pings',
        'roshan_kills',
        'rune_pickups',
        'towers_killed',
        'observer_kills',
        'sentry_kills',
        'buyback_count',
        'is_roaming'
        ]
)

In [22]:
t.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1545351 entries, 0 to 1545350
Data columns (total 45 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   match_id            1545351 non-null  int64  
 1   account_id          1545288 non-null  float64
 2   assists             1545346 non-null  float64
 3   backpack_0          1545346 non-null  float64
 4   backpack_1          1545346 non-null  float64
 5   backpack_2          1545346 non-null  float64
 6   deaths              1545346 non-null  float64
 7   denies              1545346 non-null  float64
 8   firstblood_claimed  1469565 non-null  float64
 9   gold_per_min        1545346 non-null  float64
 10  gold_spent          1491078 non-null  float64
 11  hero_damage         1491078 non-null  float64
 12  hero_healing        1491078 non-null  float64
 13  hero_id             1545346 non-null  float64
 14  item_0              1545346 non-null  float64
 15  item_1         

In [23]:
t.to_sql('players', conn, if_exists='replace', index=False)

1545351

In [24]:
t = pd.read_csv(
    '/content/merged_data/main_metadata.csv',
    usecols=[
        'match_id',
        'duration',
        'first_blood_time',
        'leagueid',
        'radiant_win',
        'start_date_time',
        'region'
        ]
)

In [25]:
t.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158479 entries, 0 to 158478
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   match_id          158479 non-null  int64  
 1   duration          158479 non-null  int64  
 2   first_blood_time  158479 non-null  int64  
 3   leagueid          158479 non-null  int64  
 4   radiant_win       157694 non-null  object 
 5   start_date_time   158479 non-null  object 
 6   region            157195 non-null  float64
dtypes: float64(1), int64(4), object(2)
memory usage: 8.5+ MB


In [26]:
t.to_sql('matches', conn, if_exists='replace', index=False)

158479

In [27]:
t = pd.read_csv(
    '/content/merged_data/picks_bans.csv',
    usecols=[
        'is_pick',
        'hero_id',
        'team',
        'order',
        'match_id',
        'leagueid'
        ]
)

In [28]:
t.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3529343 entries, 0 to 3529342
Data columns (total 6 columns):
 #   Column    Dtype  
---  ------    -----  
 0   is_pick   bool   
 1   hero_id   float64
 2   team      float64
 3   order     float64
 4   match_id  int64  
 5   leagueid  int64  
dtypes: bool(1), float64(3), int64(2)
memory usage: 138.0 MB


In [29]:
t.to_sql('picks_bans', conn, if_exists='replace', index=False)

3529343

In [30]:
t = pd.read_csv(
    '/content/merged_data/teams.csv',
    usecols=[
      'match_id',
      'leagueid',
      'radiant.team_id',
      'radiant.name',
      'radiant.tag',
      'dire.team_id',
      'dire.name',
      'dire.tag',
      ]
)

In [31]:
t.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154601 entries, 0 to 154600
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   match_id         154601 non-null  int64 
 1   leagueid         154601 non-null  int64 
 2   radiant.team_id  154601 non-null  int64 
 3   radiant.name     153258 non-null  object
 4   radiant.tag      140821 non-null  object
 5   dire.team_id     154601 non-null  int64 
 6   dire.name        153280 non-null  object
 7   dire.tag         141065 non-null  object
dtypes: int64(4), object(4)
memory usage: 9.4+ MB


In [32]:
t.to_sql('teams', conn, if_exists='replace', index=False)

154601

### Вспомогательные таблицы

In [33]:
heroes, leagues, patch, regions, items = (
    pd.read_csv('/content/Constants/Constants.Heroes.csv', index_col='Unnamed: 0').reset_index(drop=True),
    pd.read_csv('/content/Constants/Constants.Leagues.csv'),
    pd.read_csv('/content/Constants/Constants.Patch.csv'),
    pd.read_csv('/content/Constants/Constants.Regions.csv', index_col='Unnamed: 0').reset_index(drop=True),
    pd.read_csv('/content/Constants/Constants.Items.csv')
)

In [34]:
# Цикл по списку переменных
for df in [heroes, leagues, patch, regions, items]:
    # Находим имя переменной в глобальном пространстве имен
    table_name = [name for name in globals() if globals()[name] is df][0]
    # Записываем датафрейм в SQL
    df.to_sql(table_name, conn, if_exists='replace', index=False)

### items

In [35]:
pd.read_sql('select count(*) from items;', conn)

,count(*)
0,49


В таблице `items` всего 49 строк. В доте предметов гораздо больше, таблица неверная. Заменю ее корректной.

In [36]:
items_json_url = 'https://raw.githubusercontent.com/odota/dotaconstants/refs/heads/master/build/items.json'
items = pd.read_json(items_json_url, orient='index')

In [37]:
#items.memory_usage().sum()
items.info()

<class 'pandas.core.frame.DataFrame'>
Index: 459 entries, blink to diffusal_blade_2
Data columns (total 24 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   abilities    283 non-null    object 
 1   hint         283 non-null    object 
 2   id           459 non-null    int64  
 3   img          459 non-null    object 
 4   dname        450 non-null    object 
 5   qual         226 non-null    object 
 6   cost         449 non-null    float64
 7   behavior     278 non-null    object 
 8   notes        459 non-null    object 
 9   attrib       459 non-null    object 
 10  mc           459 non-null    int64  
 11  hc           457 non-null    float64
 12  cd           459 non-null    int64  
 13  lore         459 non-null    object 
 14  components   121 non-null    object 
 15  created      459 non-null    bool   
 16  charges      457 non-null    float64
 17  dmg_type     10 non-null     object 
 18  target_team  83 non-null     object 
 

In [38]:
items.head(2)

,abilities,hint,id,img,dname,qual,cost,behavior,notes,attrib,...,components,created,charges,dmg_type,target_team,target_type,dispellable,bkbpierce,desc,tier
blink,"[{'type': 'active', 'title': 'Blink', 'descrip...",[],1,/apps/dota2/images/dota_react/items/blink.png?...,Blink Dagger,component,2250.00,Point Target,Self-casting will cause you to teleport in the...,"[{'key': 'blink_range', 'value': '1200'}, {'ke...",...,None,False,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
overwhelming_blink,"[{'type': 'active', 'title': 'Overwhelming Bli...",[],600,/apps/dota2/images/dota_react/items/overwhelmi...,Overwhelming Blink,component,6800.00,Point Target,Self-casting will cause you to teleport in the...,"[{'key': 'blink_range', 'value': '1200'}, {'ke...",...,"[blink, reaver]",True,0.00,Magical,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
items = items[[
    'id',
    'dname',
    'qual',
    'cost',
    'notes',
    'mc',
    'cd',
    'lore',
    'components',
    'created',
    'charges',
    ]]

In [40]:
items = items.reset_index(names='item')

In [41]:
items['components'] = items['components'].apply(lambda x: ','.join(x) if isinstance(x, list) else x)

In [42]:
items.to_sql('items', conn, if_exists='replace', index=False)

459

In [43]:
os.path.getsize("/content/dota_pro_matches.db")

264904704

In [44]:
conn.close()

In [45]:
# Список директорий для удаления
del_list = ["Constants", "Images", "merged_data"]

# Удаление каждой директории
for folder in del_list:
    if os.path.exists(f'/content/{folder}'):
        shutil.rmtree(f'/content/{folder}')  # Удаляет директорию и все её содержимое

In [46]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [47]:
shutil.copy("/content/dota_pro_matches.db", "/content/drive/MyDrive/db/dota_pro_matches.db")

'/content/drive/MyDrive/db/dota_pro_matches.db'